In [1]:
!pip3 install contractions sentence_transformers umap-learn

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import re

import contractions
import nltk
from functools import reduce
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sentence_transformers import SentenceTransformer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import umap.umap_ as umap

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


In [3]:
def remove_tags(text: str) -> str:
    """Removes social media tags."""
    return re.sub('@\w+', '', text.lower())

def remove_punctuations(text: str) -> str:
    """Removes punctuations and keeps all alphanumerical text."""
    return re.sub(r'[^\w\s]', '', text)

def remove_stopwords(text: str) -> str:
    """Removes stopwords from text."""
    new_text = text
    for s in stopwords.words('english'):
        new_text = re.sub(f' {s} ', ' ', new_text)
    return new_text

def lemmatize(text: str) -> str:
    """Converts a text into its lemmatized form."""
    wnl = WordNetLemmatizer()
    return ' '.join([wnl.lemmatize(word) for word in text.split()])

In [4]:
functions = [
    remove_tags,
    contractions.fix,
    remove_punctuations,
    remove_stopwords,
    lemmatize, 
]

mass_apply = lambda x: reduce(lambda y, f: f(y), functions, x)

In [5]:
df = pd.read_excel('datasets/thesmartlocal.xlsx')
embedder = SentenceTransformer('all-mpnet-base-v2')


In [29]:
df['content'] = df['content'] \
    .str.replace('Follow us on Telegram for the latest updates: https://t.me/TSLMedia ', '') \
    .str.replace(" Get more stories like this. Drop us your email so you won't miss the latest news. Name Your Name Email Your Email Subscribe", '')

/var/folders/0v/f1bs90j51ss4lrwpsxqbyqk80000gq/T/ipykernel_4236/22460824.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'] \


In [ ]:
corpus_preview = list(df['preview'].apply(mass_apply))
corpus_preview_embeddings = embedder.encode(corpus_preview)

preview_embeddings_df = pd.DataFrame(corpus_preview_embeddings)
preview_embeddings_df.to_csv('/content/preview_embeddings_BERT.csv')

In [30]:
corpus_content = list(df['content'].apply(mass_apply))
corpus_content_embeddings = embedder.encode(corpus_content)

content_embeddings_df = pd.DataFrame(corpus_content_embeddings)
content_embeddings_df.to_csv('/content/content_embeddings_BERT.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/content_embeddings_BERT.csv'

In [32]:
content_embeddings_df.to_csv('./datasets/content_embeddings_BERT.csv')

In [33]:
content_embeddings_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.013740,0.022310,0.010008,0.021966,-0.007686,-0.019892,0.000366,0.016648,-0.056470,0.022251,...,0.011928,-0.023540,0.024000,0.020548,-0.018424,-0.006244,-0.001232,0.005122,0.091978,-0.020781
1,0.044542,0.039388,0.015235,-0.015298,-0.003626,0.048847,-0.023136,0.052511,-0.004008,-0.022766,...,0.022532,-0.001357,-0.030515,0.001234,-0.006508,0.004669,0.040218,0.004897,0.009139,-0.023967
2,0.050814,0.054126,-0.034730,0.035749,-0.023433,0.022131,0.013347,0.014230,-0.036089,0.005666,...,-0.023183,-0.038522,0.005266,0.022022,-0.034752,-0.014151,-0.024841,0.005025,-0.028430,-0.032074
3,0.089313,-0.010362,-0.033726,-0.010259,-0.015788,0.023985,0.100965,0.026555,-0.039481,0.057166,...,-0.000709,0.019710,-0.010780,0.004004,-0.026717,-0.015353,-0.050765,0.000482,-0.033483,-0.008738
4,0.020284,0.058323,-0.025089,0.043484,0.000672,-0.024363,-0.000483,0.029481,-0.034836,0.033536,...,0.007758,-0.033138,-0.026767,-0.004853,-0.011106,0.054159,-0.007852,-0.011077,0.062276,-0.010160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4385,0.045028,0.085607,-0.018063,0.002836,0.019220,0.023925,-0.011344,0.038834,-0.007950,-0.019925,...,0.026390,-0.039764,-0.020963,0.029887,-0.025516,-0.023160,-0.026897,-0.001962,0.028142,-0.001381
4386,-0.017006,0.031163,-0.017430,0.004410,0.004777,-0.003106,0.080405,0.014722,0.025156,-0.018668,...,0.020792,-0.023888,0.039725,-0.013932,-0.005096,0.031037,-0.018966,-0.003004,-0.017792,-0.037030
4387,0.047696,0.108562,-0.012638,-0.010616,0.021926,-0.005530,0.022787,0.059346,-0.031928,0.020405,...,0.029676,-0.042468,0.025611,0.028826,-0.030321,-0.009943,-0.017446,-0.010964,0.013120,-0.031452
4388,0.090361,0.045096,-0.030660,-0.016567,0.021249,0.025633,0.034076,0.041977,-0.029961,-0.007594,...,0.025706,-0.001486,0.035476,0.028653,0.007306,-0.022116,-0.030794,0.015195,0.003731,-0.030422


In [8]:
preview_embeddings_df = pd.read_csv('./datasets/preview_embeddings_BERT.csv')
content_embeddings_df = pd.read_csv('./datasets/content_embeddings_BERT.csv')

In [ ]:
n_components = 10
umap_model = umap.UMAP(n_components=n_components)

corpus_preview_umap = umap_model.fit_transform(corpus_preview_embeddings)
corpus_preview_umap

array([[16.70693   ,  7.820575  ,  9.057797  , ...,  8.570907  ,
        11.934839  , -0.53622854],
       [16.560272  ,  8.376318  ,  9.047758  , ...,  9.200884  ,
        12.34353   ,  0.39384997],
       [16.805668  ,  8.114635  ,  9.625596  , ...,  9.771258  ,
        12.619641  ,  0.42503592],
       ...,
       [16.891914  ,  8.716705  ,  9.708004  , ...,  8.917812  ,
        12.700037  , -0.43307   ],
       [16.70019   ,  8.110684  ,  9.87915   , ...,  9.594297  ,
        12.807452  , -0.19301294],
       [17.473438  ,  8.158591  ,  8.869604  , ...,  8.605737  ,
        13.183232  , -0.24619396]], dtype=float32)